In [1]:
#CHANGELOG
#8/14/2024 Adding averaging and image timestamping
#Changing ERA_5 to be at time of image (can try both at time of image and at time of measurement)

In [14]:
from pathlib import Path
root = Path(r'0.2_LANDSAT_extraction.ipnyb').absolute().parent.parent.parent
import pickle
with open(str(root)+r'\data\processed_data\0.1_LAGOS_processing.pickle', 'rb') as handle: #Get processed data
    dataset = pickle.load(handle)

In [15]:
LAGOS_data = dataset

In [16]:
#Extract information by location 

In [12]:
dataset = dataset[['sample_date','source_samplesite_lat_dd','source_samplesite_lon_dd']].drop_duplicates()

In [13]:
import ee
import re
ee.Authenticate()
ee.Initialize(project='lake-images-ee')

LANDSAT = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
L_Bands = LANDSAT.first().bandNames().getInfo()

p = re.compile('^ST.*')
L_Bands = [s for s in L_Bands if not p.match(s)]


ERA_5 = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR") #https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_MONTHLY_AGGR
ERA_5_bands = ["lake_mix_layer_temperature","lake_mix_layer_temperature_min","lake_mix_layer_temperature_max","lake_mix_layer_depth","lake_mix_layer_depth_min","lake_mix_layer_depth_max","u_component_of_wind_10m","u_component_of_wind_10m_min","u_component_of_wind_10m_max","v_component_of_wind_10m","v_component_of_wind_10m_min","v_component_of_wind_10m_max","total_precipitation_sum","total_precipitation_min","total_precipitation_max", "surface_net_solar_radiation_sum", "surface_net_solar_radiation_min", "surface_net_solar_radiation_max"]


In [95]:
L_Bands

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL',
 'QA_PIXEL',
 'QA_RADSAT']

In [97]:
#Add new rows

for band in ERA_5_bands:
    dataset[band] = 0

for band in L_Bands:
    dataset[band +" mean"] = 0
    dataset[band +" median"] = 0
    dataset[band +" point"] = 0

dataset["obscured"]=0
dataset["image_id"]=0

import re


In [21]:
import numpy as np
split = np.array_split(dataset, 100)
for i in range(100):
    split[i] = split[i].dropna(axis=0) #drop

C:\Users\Saltp\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [22]:


import pandas as pd

import calendar
import re

import math



from datetime import datetime, timedelta
from dateutil import parser

def add_masks(image):
    QA_pixel_at_point = image.select('QA_PIXEL')

    SR_QA_at_point = image.select('SR_QA_AEROSOL')
    

    not_cloud_and_cloud_shadow_and_frozen = QA_pixel_at_point.bitwiseAnd(ee.Number(8+16+32)).eq(0)
 #Could technichally do in one bitwiseAnd
    is_water =  SR_QA_at_point.bitwiseAnd(ee.Number(4))
    
    return image.updateMask(is_water).updateMask(not_cloud_and_cloud_shadow_and_frozen)

def is_obscured(image, roi, threshold, measurement_point): #threshold is percent umasked
    #Total num of pixels
    total=image.unmask().reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=roi,
    scale=30
    ).get('SR_B1')
    unmasked = image.reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=roi,
    scale=30).get('SR_B1')

    is_unmasked = ee.Number(unmasked).divide(ee.Number(total)).lte(threshold)

    measurement_pixel_good = ee.Number(image.mask().reduceRegion(ee.Reducer.first(), measurement_point).get('SR_B1')).eq(0)

    #Each is equal to 0 if and only if point is good
    
    #Check if point and region are both good
    image = image.set('obscured',is_unmasked.add(measurement_pixel_good))
    
    return image

def closest_to_measurement(image, measurement_date):
    measurement_date = ee.Date(measurement_date)
    difference = measurement_date.difference(image.date(),'hours').abs()
    image = image.set('time_to_measurement',difference)
    return image
    

def extract_ERA5_and_LANDSAT_info(measurement, radius):


#TODO:
#Set up radius and weather time as parameter
#Set up image timestamp
#Select image closest temporally to measurement
#Redo get_cloudyness


 #---   
    threshold = 0.3 #Cut image if less than 0.3 visible
    #Filter location and time to containing circle of radius centered at measuremnt and in month containing measurment
    #Don't get 15 days in both directions to match behavior of chla-predictor
    date = measurement.sample_date
    date = parser.parse(date).date()
    measurement_point = ee.Geometry.Point([measurement.source_samplesite_lon_dd,measurement.source_samplesite_lat_dd])
    measurement_region = ee.Geometry.Point([measurement.source_samplesite_lon_dd,measurement.source_samplesite_lat_dd]).buffer(radius)

    first = date - timedelta(days = 15)
    last = date + timedelta(days = 15)
    imagecol = LANDSAT.filterDate(ee.Date(str(first)),ee.Date(str(last))).filterBounds(measurement_region) #Filter images in month and location


    #Filter clouds, cloud shadow, ice, land
    imagecol = imagecol.map(lambda x: x.clip(measurement_region))
    imagecol = imagecol.map(lambda x: add_masks(x))
    imagecol = imagecol.map(lambda x: is_obscured(x, measurement_region, threshold, measurement_point))
    imagecol = imagecol.filter(ee.Filter.eq('obscured',0))

    
    if(imagecol.size().getInfo()==0):
        measurement["obscured"]=1
    else:
        #Select image temporally closest to measurement
        imagecol = imagecol.map(lambda x: closest_to_measurement(x,str(date)))
        image = imagecol.sort('time_to_measurement').first()


        #get relevant image data (mean + median of bands) + id + timestamp
        mean = image.reduceRegion(ee.Reducer.mean(), measurement_region)
        median = image.reduceRegion(ee.Reducer.median(), measurement_region)    
        point = image.reduceRegion(ee.Reducer.first(), measurement_point)
        for band in L_Bands:
            measurement[band + " mean"] = mean.get(band).getInfo()
            measurement[band + " median"] = median.get(band).getInfo()
            measurement[band + " point"] = point.get(band).getInfo()

        measurement["image_id"] = image.id().getInfo()

        
        #get relevant ERA_5 data (always measurement date)
        measurement_date = ee.Date(measurement.sample_date)
        ERA_measurement = ERA_5.filterDate(measurement_date).first()

        try:
        
            ERA_image_value = ERA_measurement.reduceRegion(ee.Reducer.first(),ee.Geometry.Point([measurement.source_samplesite_lon_dd,measurement.source_samplesite_lat_dd]))
            
            for band in ERA_5_bands:
                measurement[band] = ERA_image_value.get(band).getInfo()
        except:
            measurement["lake_mix_layer_temperature"]=-10000 #Error

    return measurement
            
            

In [9]:
result = dataset.iloc[0:100].apply(lambda x: extract_ERA5_and_LANDSAT_info(x, 400), axis = 1)

In [10]:
result

,sample_date,source_samplesite_lat_dd,source_samplesite_lon_dd,lake_mix_layer_temperature,lake_mix_layer_temperature_min,lake_mix_layer_temperature_max,lake_mix_layer_depth,lake_mix_layer_depth_min,lake_mix_layer_depth_max,u_component_of_wind_10m,...,SR_QA_AEROSOL median,SR_QA_AEROSOL point,QA_PIXEL mean,QA_PIXEL median,QA_PIXEL point,QA_RADSAT mean,QA_RADSAT median,QA_RADSAT point,obscured,image_id
0,2005-09-07,42.140963,-72.120217,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0,0.000000,0.0,0,0,0,0,1,0
2,2005-08-10,42.698886,-71.001423,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0,0.000000,0.0,0,0,0,0,1,0
4,1999-07-08,42.045000,-72.753000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0,0.000000,0.0,0,0,0,0,1,0
5,2000-07-08,42.045000,-72.753000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0,0.000000,0.0,0,0,0,0,1,0
6,2000-07-08,42.029000,-72.759000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0,0.000000,0.0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,2015-08-18,47.234775,-91.822411,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0,0.000000,0.0,0,0,0,0,1,0
256,2015-09-22,47.234775,-91.822411,287.241862,286.558838,289.067627,25.035889,17.056641,28.78125,1.597263,...,100.0,100,21924.281957,21952.0,21952,0,0,0,0,LC08_027027_20150922
261,2016-05-24,47.234775,-91.822411,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0,0.000000,0.0,0,0,0,0,1,0
265,2016-06-14,47.234775,-91.822411,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0,0.000000,0.0,0,0,0,0,1,0


In [11]:
result[result.obscured==0]

,sample_date,source_samplesite_lat_dd,source_samplesite_lon_dd,lake_mix_layer_temperature,lake_mix_layer_temperature_min,lake_mix_layer_temperature_max,lake_mix_layer_depth,lake_mix_layer_depth_min,lake_mix_layer_depth_max,u_component_of_wind_10m,...,SR_QA_AEROSOL median,SR_QA_AEROSOL point,QA_PIXEL mean,QA_PIXEL median,QA_PIXEL point,QA_RADSAT mean,QA_RADSAT median,QA_RADSAT point,obscured,image_id
22,2017-07-08,42.282000,-71.427000,293.416423,292.170166,294.065674,8.116089,7.414062e+00,10.061523,1.540179,...,164.0,164,21932.039920,21952.0,21952,0,0,0,0,LC08_012031_20170716
108,2014-11-24,41.176146,-74.437597,273.461751,273.149658,274.583252,1.657471,-7.105434e-15,4.000000,-0.045373,...,164.0,228,21937.037055,21952.0,21952,0,0,0,0,LC08_014031_20141111
113,2017-06-06,41.183565,-74.432169,288.342570,288.042236,288.587158,4.000000,4.000000e+00,4.000000,-1.387932,...,228.0,228,21879.722246,21824.0,21824,0,0,0,0,LC08_014031_20170527
256,2015-09-22,47.234775,-91.822411,287.241862,286.558838,289.067627,25.035889,1.705664e+01,28.781250,1.597263,...,100.0,100,21924.281957,21952.0,21952,0,0,0,0,LC08_027027_20150922


In [12]:
split[55][split[55].sample_date=='2002-06-04']

,sample_date,source_samplesite_lat_dd,source_samplesite_lon_dd,lake_mix_layer_temperature,lake_mix_layer_temperature_min,lake_mix_layer_temperature_max,lake_mix_layer_depth,lake_mix_layer_depth_min,lake_mix_layer_depth_max,u_component_of_wind_10m,...,SR_QA_AEROSOL median,SR_QA_AEROSOL point,QA_PIXEL mean,QA_PIXEL median,QA_PIXEL point,QA_RADSAT mean,QA_RADSAT median,QA_RADSAT point,obscured,image_id
953178,2002-06-04,39.148337,-89.878616,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
955226,2002-06-04,37.778836,-89.449500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:

import dask.dataframe as dd

import pickle

print(datetime.now())
r=400 #H: If r to small ERA5 doesn't work #Yup thats what it is..

def f(x):
    return extract_ERA5_and_LANDSAT_info(x,r)
for i in range(97,100):
    chunk = split[i]
    ddata = dd.from_pandas(chunk, npartitions=10) #maybe slow down, exceeded quota for project
    res = ddata.map_partitions(lambda df: df.apply(f, axis=1), meta=pd.DataFrame(dtype=object, columns = dataset.columns)).compute(scheduler='threads')

    name_string = "sat_dat_chunk_"+str(i)+".pickle"
    with open(name_string, 'wb') as handle:
        pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(i)
print(datetime.now())

C:\Users\Saltp\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


2024-09-03 07:57:32.029630


C:\Users\Saltp\AppData\Local\Temp\ipykernel_19292\966757336.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measurement[band + " mean"] = mean.get(band).getInfo()
C:\Users\Saltp\AppData\Local\Temp\ipykernel_19292\966757336.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measurement[band + " median"] = median.get(band).getInfo()
C:\Users\Saltp\AppData\Local\Temp\ipykernel_19292\966757336.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

97
98
99
2024-09-03 08:53:02.836899


In [ ]:
LANDSAT.filterDate(ee.Date('2020-02-02')).getInfo()['constantValue']

In [ ]:
#Without internal imports ~30 sec for 100

In [ ]:
#5 SECONDS FOR 100 LETS GOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
# OMG THAT'S LIKE A really fast

In [27]:
dataset.shape

(1027021, 53)

In [99]:
import pickle
import pandas as pd
df = 0
with open(r'sat_dat_chunk_0.pickle', 'rb') as handle: #NEVER OVERWRITE CHUNKS, THEY ARE BACKUP
        df = pickle.load(handle)
for i in range(1,100):
    with open(r'sat_dat_chunk_'+str(i)+'.pickle', 'rb') as handle:
        r = pickle.load(handle)
        df=pd.concat([df,r])


In [100]:
str(root)

'C:\\Users\\Saltp\\OneDrive\\Desktop\\Algal Bloom Prediction\\Pipeline'

In [101]:
with open(str(root)+r'\data\processed_data\0.2.0_LANDSAT_extraction.pickle', 'wb') as handle:
    pickle.dump(df, handle)

In [1]:
import pickle
from pathlib import Path
root = Path(r'0.2_LANDSAT_extraction.ipnyb').absolute().parent.parent.parent
with open(str(root)+r'\data\processed_data\0.2.0_LANDSAT_extraction.pickle', 'rb') as handle:
    df = pickle.load(handle)


In [3]:
df

,sample_date,source_samplesite_lat_dd,source_samplesite_lon_dd,lake_mix_layer_temperature,lake_mix_layer_temperature_min,lake_mix_layer_temperature_max,lake_mix_layer_depth,lake_mix_layer_depth_min,lake_mix_layer_depth_max,u_component_of_wind_10m,...,SR_QA_AEROSOL median,SR_QA_AEROSOL point,QA_PIXEL mean,QA_PIXEL median,QA_PIXEL point,QA_RADSAT mean,QA_RADSAT median,QA_RADSAT point,obscured,image_id
0,2005-09-07,42.140963,-72.120217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0,0.0,0,0,1,0
2,2005-08-10,42.698886,-71.001423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0,0.0,0,0,1,0
4,1999-07-08,42.045000,-72.753000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0,0.0,0,0,1,0
5,2000-07-08,42.045000,-72.753000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0,0.0,0,0,1,0
6,2000-07-08,42.029000,-72.759000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0,0.0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2108447,2021-08-03T15:06Z,47.159090,-99.113880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0,0.0,0,0,1,0
2108448,2021-09-07T14:36Z,47.159090,-99.113880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0,0.0,0,0,1,0
2108449,2021-09-14T13:34Z,47.159090,-99.113880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0,0.0,0,0,1,0
2108450,2021-10-12T13:22Z,47.159090,-99.113880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0,0.0,0,0,1,0


In [10]:
df = df[(df.obscured!=1) & (df["lake_mix_layer_temperature"]!=-10000)]

In [20]:
#Merge with original data
import pandas as pd
dataset = pd.merge(df,LAGOS_data, how = 'inner', on = ['sample_date','source_samplesite_lat_dd','source_samplesite_lon_dd'], validate='one_to_many')

In [21]:
dataset = dataset.dropna(axis = 0)

In [22]:
dataset.columns

Index(['sample_date', 'source_samplesite_lat_dd', 'source_samplesite_lon_dd',
       'lake_mix_layer_temperature', 'lake_mix_layer_temperature_min',
       'lake_mix_layer_temperature_max', 'lake_mix_layer_depth',
       'lake_mix_layer_depth_min', 'lake_mix_layer_depth_max',
       'u_component_of_wind_10m', 'u_component_of_wind_10m_min',
       'u_component_of_wind_10m_max', 'v_component_of_wind_10m',
       'v_component_of_wind_10m_min', 'v_component_of_wind_10m_max',
       'total_precipitation_sum', 'total_precipitation_min',
       'total_precipitation_max', 'surface_net_solar_radiation_sum',
       'surface_net_solar_radiation_min', 'surface_net_solar_radiation_max',
       'SR_B1 mean', 'SR_B1 median', 'SR_B1 point', 'SR_B2 mean',
       'SR_B2 median', 'SR_B2 point', 'SR_B3 mean', 'SR_B3 median',
       'SR_B3 point', 'SR_B4 mean', 'SR_B4 median', 'SR_B4 point',
       'SR_B5 mean', 'SR_B5 median', 'SR_B5 point', 'SR_B6 mean',
       'SR_B6 median', 'SR_B6 point', 'SR_B7 mean'

In [23]:
dataset.shape

(73625, 66)

In [28]:
dict(dataset.isna().any())

{'sample_date': False,
 'source_samplesite_lat_dd': False,
 'source_samplesite_lon_dd': False,
 'lake_mix_layer_temperature': False,
 'lake_mix_layer_temperature_min': False,
 'lake_mix_layer_temperature_max': False,
 'lake_mix_layer_depth': False,
 'lake_mix_layer_depth_min': False,
 'lake_mix_layer_depth_max': False,
 'u_component_of_wind_10m': False,
 'u_component_of_wind_10m_min': False,
 'u_component_of_wind_10m_max': False,
 'v_component_of_wind_10m': False,
 'v_component_of_wind_10m_min': False,
 'v_component_of_wind_10m_max': False,
 'total_precipitation_sum': False,
 'total_precipitation_min': False,
 'total_precipitation_max': False,
 'surface_net_solar_radiation_sum': False,
 'surface_net_solar_radiation_min': False,
 'surface_net_solar_radiation_max': False,
 'SR_B1 mean': False,
 'SR_B1 median': False,
 'SR_B1 point': False,
 'SR_B2 mean': False,
 'SR_B2 median': False,
 'SR_B2 point': False,
 'SR_B3 mean': False,
 'SR_B3 median': False,
 'SR_B3 point': False,
 'SR_B4 mean

In [30]:
with open(str(root)+r'\data\processed_data\0.2_LANDSAT_extraction.pickle', 'wb') as handle:
    pickle.dump(dataset, handle)


In [7]:
loc = (dat.source_samplesite_lon_dd[1000],dat.source_samplesite_lat_dd[1000])

,lagoslakeid,lake_nhdid,lake_nhdfcode,lake_nhdftype,lake_reachcode,lake_namegnis,lake_namelagos,lake_onlandborder,lake_ismultipart,lake_missingws,...,hu8_zoneid,hu4_zoneid,county_zoneid,state_zoneid,epanutr_zoneid,omernik3_zoneid,wwf_zoneid,mlra_zoneid,bailey_zoneid,neon_zoneid
174487,211,{492CB89C-D0FF-47E3-A144-39464508B9FD},39004,390,4.150408e+12,Lake Champlain,Lake Champlain; Devils Pond,N,Y,N,...,hu8_81,hu4_5,county_654,state_5,epanutr_2,omernik3_84,wwf_5,mlra_71,bailey_5,neon_1


     sample_date  source_samplesite_lat_dd  source_samplesite_lon_dd  \
653   2013-05-02                   44.4711                  -73.2992   
654   2013-05-02                   44.4711                  -73.2992   
655   2013-05-02                   44.4711                  -73.2992   
656   2013-05-02                   44.4711                  -73.2992   
657   2013-05-07                   44.5819                  -73.2811   
...          ...                       ...                       ...   
1321  2020-09-22                   44.4711                  -73.2992   
1322  2020-09-22                   44.4711                  -73.2992   
1323  2020-09-23                   44.4747                  -73.2317   
1324  2020-09-23                   44.4747                  -73.2317   
1325  2020-09-23                   44.4747                  -73.2317   

      lake_mix_layer_temperature  lake_mix_layer_temperature_min  \
653                   279.521200                      279.316650   

In [11]:
dat.iloc[1000]

sample_date                          2017-05-16
source_samplesite_lat_dd                44.4258
source_samplesite_lon_dd               -73.2319
lake_mix_layer_temperature           282.119059
lake_mix_layer_temperature_min       281.952393
                                      ...      
lake_lon_decdeg                      -73.420171
lake_elevation_m                           28.9
lake_totalarea_ha                 121898.728049
lake_perimeter_m                  746740.783691
lake_shorelinedevfactor                6.423814
Name: 1015, Length: 66, dtype: object

In [12]:
import geemap

map_l8 = geemap.Map(center=[37.5010, -122.1899], zoom=10)
map_l8.centerObject(img)
L8_754_viz = {'bands':['SR_B7', 'SR_B5', 'SR_B4'], 'min':1, 'max':65455}
# Add the image layer to the map and display it.
map_l8.add_layer(img, L8_754_viz, 'img')
map_l8.add_layer(ee.Geometry.Point(loc),{'color':'red'}, 'samp')

display(map_l8)

Map(center=[37.501, -122.1899], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…